In [3]:
import pandas as pd
import json

In [10]:
with open("../new-york-open-data-explorer/src/datasets.json",'r') as f:
    datasets = json.load(f)

In [11]:
len(datasets)

2804

In [30]:
len(set([2,4,5,6,7]).intersection(set([2,5])))

2

In [47]:
def find_joinable(dataset):
    columns = set( [ a.lower() for a in dataset['resource']['columns_field_name']])
    result = []
    for comp_dataset in datasets:
        if(comp_dataset['resource']['id'] == dataset['resource']['id']):
            continue
        comp_columns = set([a.lower() for a in comp_dataset['resource']['columns_field_name']])
        no_matches = len(columns.intersection(comp_columns))
        if(no_matches>0):
            result.append({'o': dataset['resource']['id'],
                           'd':comp_dataset['resource']['id'],
                           'col':no_matches
                          })
        
    return result

In [48]:
joins = []
for dataset in datasets:
    joins.extend(find_joinable(dataset))

In [126]:
joins = pd.DataFrame(joins)

In [ ]:
number_lookup = 

In [127]:
joins

,o,d,col
0,fuzi-xuv4,dggd-3jfu,1
1,fuzi-xuv4,p5yk-zjsc,6
2,fuzi-xuv4,v2hd-uwxx,3
3,fuzi-xuv4,i6ds-scnc,1
4,fuzi-xuv4,cirn-n726,8
...,...,...,...
759681,w9ak-ipjd,867j-5pgi,8
759682,w9ak-ipjd,uvpi-gqnh,7
759683,w9ak-ipjd,ymhw-9cz9,7
759684,w9ak-ipjd,4e2n-s75z,9


In [55]:
potential_join_datasets = pd.DataFrame(joins).groupby('o').count()[['d']]

In [56]:
potential_join_datasets

,d
o,
2232-dj5q,210
22rr-ujq3,324
23z9-6uk9,1101
24nr-gahi,233
259a-b6s7,29
...,...
yz7z-iupz,450
zmut-au2w,654
zpd4-gad8,858


In [65]:
departmentList = [d['resource']['domain_metadata']['Dataset-Information_Agency'] for d in datasets]

KeyError: 'domain_metadata'

In [88]:
def get_agency(dataset):
    metadata = dataset['classification']['domain_metadata']
    hits= [entry['value'] for entry in metadata if entry['key'] == 'Dataset-Information_Agency']
    if(len(hits)>0):
        return hits[0]
    return None

In [91]:
data = [ {"id": dataset['resource']['id'], 
   "page_views" : dataset['resource']['page_views']['page_views_total'],
   "downloads": dataset['resource']['download_count'],
   "columns" : len(dataset['resource']['columns_name']),
   'agency': get_agency(dataset)
  }
 for dataset in datasets]

In [100]:
result = (pd.merge(pd.DataFrame(data), 
                  potential_join_datasets, 
                  left_on='id', 
                  right_index=True, 
                  how='left')
           .fillna(0))
result = result.assign(agency=result.agency.astype('category'))

In [110]:
import seaborn as sns

In [113]:
qualitative_colors = sns.color_palette("Set3", 30)
qualitative_colors

[(0.5529411764705883, 0.8274509803921568, 0.7803921568627451),
 (1.0, 1.0, 0.7019607843137254),
 (0.7450980392156863, 0.7294117647058823, 0.8549019607843137),
 (0.984313725490196, 0.5019607843137255, 0.4470588235294118),
 (0.5019607843137255, 0.6941176470588235, 0.8274509803921568),
 (0.9921568627450981, 0.7058823529411765, 0.3843137254901961),
 (0.7019607843137254, 0.8705882352941177, 0.4117647058823529),
 (0.9882352941176471, 0.803921568627451, 0.8980392156862745),
 (0.8509803921568627, 0.8509803921568627, 0.8509803921568627),
 (0.7372549019607844, 0.5019607843137255, 0.7411764705882353),
 (0.8, 0.9215686274509803, 0.7725490196078432),
 (1.0, 0.9294117647058824, 0.43529411764705883),
 (0.5529411764705883, 0.8274509803921568, 0.7803921568627451),
 (1.0, 1.0, 0.7019607843137254),
 (0.7450980392156863, 0.7294117647058823, 0.8549019607843137),
 (0.984313725490196, 0.5019607843137255, 0.4470588235294118),
 (0.5019607843137255, 0.6941176470588235, 0.8274509803921568),
 (0.9921568627450981,

In [121]:
qualitative_colors = sns.color_palette("Set3", 30)
color_list = dict(zip(
list(result.groupby('agency').count().sort_values(by='page_views', ascending=False).head(30).index),qualitative_colors,))

In [123]:
result = result.assign(r= result.agency.apply(lambda x: color_list[x][0] if x in color_list else 0.7 ),
             g= result.agency.apply(lambda x: color_list[x][1] if x in color_list else 0.7 ),
             b =result.agency.apply(lambda x: color_list[x][2] if x in color_list else 0,7))

In [124]:
result.to_csv('static/dataset_stats.csv',index=False)

In [90]:
get_agency(datasets[10])

'New York City Housing Authority (NYCHA)'

In [74]:
datasets[10]['classification']['domain_metadata']

[{'key': 'Data-Collection_Data-Collection',
  'value': 'Services available to NYCHA Residents - Local Law 163'},
 {'key': 'Update_Automation', 'value': 'No'},
 {'key': 'Update_Date-Made-Public', 'value': '11/29/2019'},
 {'key': 'Update_Update-Frequency', 'value': 'Annually'},
 {'key': 'Dataset-Information_Agency',
  'value': 'New York City Housing Authority (NYCHA)'}]

In [133]:
pos_lookup = dict(zip(list(result.id), list(result.index)))

In [140]:
joins.assign(o = joins.o.apply(lambda x: pos_lookup[x]),
             d = joins.d.apply(lambda x: pos_lookup[x])).to_csv('static/links.csv', index=False)